In [1]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

ModuleNotFoundError: ignored

## Problem 1. Profit Maximization

### Parameters
- $d_{i,j}$: The distance between station $i$ and station $j$.
- $E_i$: The existing number of bikes at station $i$ before rebalancing.
- $I_i = [\lambda_i - \omega_i, \lambda_i + \omega_i]$: The ideal threshold of bikes at station $i$ after rebalancing, which is represented by a predetermined idea amount $\lambda_i$ associated with a symmetric threshold of $2\omega_i$.
- $\gamma_i$: Capacity (number of docks) of bike station $i$.
- $n$: Number of stations.
- $C$: Capacity of the truck.
- $w$: Wage for driver, dollars per bike.
- $h$: Disinfecting cost, dollars per bike.
- $g$: Price of Gasoline, dollars per kilometer.
- $R$: The predicted daily revenue.
- $s_{i}^+$: A binary value that indicates whether there is a bike surplus at station $i$, i.e. $s_{i}^+ = 1$ when $E_i > \lambda_i + \omega_i$, and zero otherwise.
- $s_{i}^-$: A binary value that indicates whether there is a bike shortage at station $i$, i.e. $s_{i}^- = 1$ when $E_i < \lambda_i - \omega_i$, and zero otherwise.

In [ ]:
#Data importing and preprocessing
Verdun_stations = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/Verdun_stations.csv')
Verdun_dock_capacity = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/Verdun_dock_capacity.csv')

Verdun_dock_capacity = Verdun_dock_capacity.drop(columns=['Unnamed: 0'])
Verdun_stations = Verdun_stations.drop(columns=['Unnamed: 0'])

Verdun_dock_capacity = Verdun_dock_capacity.rename(columns = {'station_code': 'Code', 'dock capacity': 'capacity'})
Verdun_stations = Verdun_stations.join(Verdun_dock_capacity.set_index('Code'), on = 'Code')

n = Verdun_stations.shape[0]
#stations = [coordinates of each station]
stations = [[np.array(Verdun_stations.latitude)[i], np.array(Verdun_stations.longitude)[i]] for i in range(n)]

In [ ]:
Verdun_net_balance20190708 = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/net_balance2019-07-08.csv')
Verdun_net_balance20190708 = Verdun_net_balance20190708.drop(columns=['Unnamed: 0'])

#Use the demand of '7/8/19 8:00' as an example
diff = np.array(Verdun_net_balance20190708['7/8/19 8:00'])

In [ ]:
cap = np.array(Verdun_stations['capacity'])
ideal_level = np.floor(cap * 0.7) #ideal level is at 70%
ideal_level = ideal_level.astype(int)

#Existing number of bikes at stations, list of integers of length n
E = np.floor(cap * 0.7) + diff
E = E.astype(int)

#Manhattan Distance
#@ith WGS 1984 coordinate projection 111, 1 degree = 111 KM
d_m = [[111 * abs(i[0] - j[0]) + 111 * abs(i[1] - j[1]) for i in stations] for j in stations]

threshold = 3
I = [[ideal_level[i] - threshold, ideal_level[i] + threshold] for i in range(n)]


#Capacity of the truck
C = 40

#Binary values indicates surplus/shortage at staion i
s1 = [int(E[i] - I[i][1] > 0) for i in range(n)] #surplus
s2 = [int(I[i][0] - E[i] > 0) for i in range(n)] #shortage

#Obj parameters: wages and costs
h = 0.875
g = 0.294
R = 1000

### Variables
- $x_{i,j}$: A binary variable that indicates the relocation truck passes through station $i$ to station $j$ if $x_{i,j} = 1$, and zero otherwise.
- $c_{i}$: The current number of bikes on the truck before arriving at station $i$.
- $d_{i}$: The number of bikes the truck drops off at station $i$.
- $p_{i}$: The number of bikes the truck picks up at station $i$.

### Dummy Variables
- $z_i$: Non-decreasing variables to ensure continuity of the route, for $i \in \{1 \dots n\}$.


### Other Notations
- $B$: The total number of bikes that are relocated.
- $D$: The total distance that the truck has traveled.
- $V$: The total number of stations visited (for record tracking purpose ONLY).

In [ ]:
#Profit Maximization Model
prob = Model('BIXI')

x = prob.addVars(n, n, vtype='B', name = 'x') #BINARY: x_{i,j}
c = prob.addVars(n,lb = 0, vtype='I', name = 'c') #INTEGER: c_i 
d = prob.addVars(n,lb = 0, vtype='I', name = 'd') #INTEGER: d_i
p = prob.addVars(n,lb = 0, vtype='I', name = 'p') #INTEGER: p_i
z = prob.addVars(n,lb = 0, name = 'z')

B = prob.addVar(lb = 0, vtype='I', name = 'B') #INTEGER: total number of bikes rebalanced
D = prob.addVar(lb = 0, name = 'D') #INTEGER: total distance travelled by truck

#Variable to check accuracy
V = prob.addVar(lb = 0, vtype='I', name = 'V') #INTEGER, total number of stations visited
d_sum = prob.addVar(lb = 0, vtype='I', name = 'd_sum') #INTEGER, total number of bikes dropped off
p_sum = prob.addVar(lb = 0, vtype='I', name = 'p_sum') #INTEGER, total number of bikes picked up

### Assumptions
- Routing
  - 1. Each station is visited at most once. (Routing-1)
  - 2. The truck starts at station 0 and ends at station 0. (Routing-2)
  - 3. The distance will be calcuated using Manhattan distance.
- Rebalancing
  - 1. The truck arrives at station 0 with 20 bikes at the beginning of each working hour.
  - 2. The operational cost has three components, the wage for the worker, gas cost and disinfecting cost.

### Variations
- fixed subpath on bike stations located near hospitals, extra cost for disinfecting during COVID-19
- Nonlinear objective

### Objective
$$\text{Minimize } \big( h * B + g * D + w\big)$$

In [ ]:
obj = h * B + g * D + 20.52
prob.setObjective(obj, GRB.MINIMIZE)

### Constraints - Routing

$$\begin{align}
\sum_i x_{ij} & \le 1 \hspace{0.5cm} i = \{2 \dots n\} \tag{Routing-1.1}\\
\sum_j x_{ij} & \le 1 \hspace{0.5cm} j = \{2 \dots n\} \tag{Routing-1.2}\\
\sum_i x_{ij} & = \sum_i x_{ji} \hspace{0.5cm} \tag{Routing-1.3}\\
\sum_i x_{ii} & = 0 \hspace{0.5cm} \tag{Routing-1.4}\\
\sum_i x_{i1} & = 1 \tag{Routing-2.1}\\
\sum_j x_{1j} & = 1 \tag{Routing-2.2}\\
\sum_{i,j} x_{ij} d_{ij} & = D  \tag{Total distance}\\
z_j & \ge z_i + 1 - 1996(1 - x_{ij}) \tag{Continuity of route}\\
\end{align}$$

In [ ]:
#---Routing---

#Routing-1.1
for j in range(n):
    prob.addConstr(quicksum(x[i,j] for i in range(n)) <= 1)
    
#Routing-1.2    
for i in range(n):
    prob.addConstr(quicksum(x[i,j] for j in range(n)) <= 1)

#Routing-1.3
for i in range(n):
    prob.addConstr(quicksum(x[i,j] for j in range(n)) == quicksum(x[j,i] for j in range(n))) 

#Routing-1.4
for i in range(1,n):
    prob.addConstr(x[i,i] == 0)    
    
#Routing-2.1
# prob.addConstr(quicksum(x[i,0] for i in range(n)) == 1)  

#Routing-2.2
# prob.addConstr(quicksum(x[0,i] for i in range(n)) == 1)

#Total Distance
prob.addConstr(quicksum(x[i,j] * d_m[i][j] for i in range(n) for j in range(n)) == D)

#Route Continuity
#Avoid sub trips by implementing a non-decreasing variable z
#When the truck travels from i to j, z[j] > z[i]
for i in range(n):
    for j in range(1,n):
        prob.addConstr(z[j] >= z[i] + 1 - 1996 * (1 - x[i,j]))

### Constraints - Rebalancing(working in progress)

$$\begin{align}
c_i & \le C \tag{Capacity of the truck}\\
s_i^- d_i + s_i^+ p_i & = 0 \tag{Pick up XOR Drop off at any station}\\
p_i & \le E_i \tag{Pick up within ideal threshold}\\
\sum_i d_i & \le \sum_i p_i \tag{Drop off LESS THAN pick up}\\
\sum_i d_i + p_i & = B \tag{Total amount of bikes rebalanced}\\
d_j + p_j & \ge \sum_i x_{ij} \tag{A station is visted when necessary}\\
c_j + d_j + p_j + \alpha_j & \le 1996\sum_i x_{ij} \tag{A station is visted when necessary}\\
c_j & \ge c_i - d_i + p_i - C(1 - x_{ij}) \hspace{0.5cm} \tag{Continuity of truck inventory - 1}\\
c_j & \le c_i - d_i + p_i - C(1 - x_{ij}) \hspace{0.5cm} \tag{Continuity of truck inventory - 2}\\
\end{align}$$

In [ ]:
#---Rebalancing---     
        
#Truck arrives at station 1 with 20 bikes
prob.addConstr(c[0] == 20)


for i in range(1,n):
    #Pick up XOR Drop off at any station
    prob.addConstr(s1[i] * d[i] + s2[i] * p[i] == 0)

    #Capacity of the truck
    prob.addConstr(c[i] <= C)
    
    #pick up within ideal threshold
    prob.addConstr(p[i] <= E[i])
    prob.addConstr(p[i] >= E[i] - I[i][1])
    
    #drop off within ideal threshold
    prob.addConstr(d[i] <= cap[i] - E[i])
    prob.addConstr(d[i] >= (I[i][0] - E[i])) #nonbinding when surplus  
    
#Total amount of bikes rebalanced
prob.addConstr(quicksum(p[i] + d[i] for i in range(n)) == B)

#A station is visted when necessary (synergy with the route)
for i in range(1,n):
    prob.addConstr(d[i] + p[i] >= quicksum(x[j,i] for j in range(n)))
    #when station i is not visited LHS = RHS = 0
    #when station i is visited 
    prob.addConstr(c[i] + d[i] + p[i] + z[i] <= 2 * C * quicksum(x[j,i] for j in range(n)))
                   

#Continuity of truck inventory
for i in range(n):
    for j in range(1,n):
        prob.addConstr(c[j] <= c[i] - d[i] + p[i] + C * (1 - x[i,j]))
        prob.addConstr(c[j] >= c[i] - d[i] + p[i] - C * (1 - x[i,j]))

In [ ]:
#---Variables for record tracking---           
prob.addConstr(quicksum(x[i,j] for i in range(n) for j in range(n)) == V)    
prob.addConstr(quicksum(d[i] for i in range(n)) == d_sum)
prob.addConstr(quicksum(p[i] for i in range(n)) == p_sum)

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
prob.optimize()
prob.printAttr('X')

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1884 rows, 401 columns and 10860 nonzeros
Model fingerprint: 0xc7fcbf27
Variable types: 19 continuous, 382 integer (324 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+03]
  Objective range  [3e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]

MIP start from previous solve produced solution with objective 27.4049 (0.01s)
Loaded MIP start from previous solve with objective 27.4049

Presolve removed 943 rows and 34 columns
Presolve time: 0.04s
Presolved: 941 rows, 367 columns, 5119 nonzeros
Variable types: 17 continuous, 350 integer (304 binary)

Root relaxation: objective 2.673572e+01, 42 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   26.73572    0  

In [ ]:
print('Operation cost for July 8th 8:00AM - 9:00AM: ' + str(round(obj.getValue(),2)))
print('The driver travelled {} stations and rebalanced {} bikes (picked up {} bikes and dropped off {} bikes).'.format(V.x, B.x, p_sum.x, d_sum.x))

Operation cost for July 8th 8:00AM - 9:00AM: 27.4
The driver travelled 5.0 stations and rebalanced 6.0 bikes (picked up 2.0 bikes and dropped off 4.0 bikes).


### Now we wrap everything into one function, which automatically updates the number of bikes at each station after riders pick up, drop off and the rebalancing, and simulate the rebalancing for July 8th.

In [ ]:
def hour_rebalance(df):
    
    bikes_rebalanced = []
    before = []
    after = []
    visits = []
    costs = []
    
    n = len(stations)
    
    daily_cost = 0
    bikes = 0
    
    C = 40
    d_m = [[111 * abs(i[0] - j[0]) + 111 * abs(i[1] - j[1]) for i in stations] for j in stations]
    
    cap = np.array(Verdun_stations['capacity'])
    ideal_level = np.floor(cap * 0.7)
    E = ideal_level
    
    threshold = 3
    I = [[ideal_level[i] - threshold, ideal_level[i] + threshold] for i in range(n)]
    
    #Obj parameters: wages and costs
    h = 0.875
    g = 0.294
    R = 1000

    for col in list(df.columns)[2:14]:
        diff = np.array((df[col]))
        E = E + diff
        before.append(E)
        print(col)
        print(E)

        #Binary values indicates surplus/shortage at staion i
        s1 = [int(E[i] - I[i][1] > 0) for i in range(n)] #surplus
        s2 = [int(I[i][0] - E[i] > 0) for i in range(n)] #shortage

        
        prob = Model('BIXI')

        x = prob.addVars(n, n, vtype='B', name = 'x') #BINARY
        c = prob.addVars(n,lb = 0, vtype='I', name = 'c') #INTEGER
        d = prob.addVars(n,lb = 0, vtype='I', name = 'd') #INTEGER
        p = prob.addVars(n,lb = 0, vtype='I', name = 'p') #INTEGER
        z = prob.addVars(n,lb = 0, name = 'z')

        B = prob.addVar(lb = 0, vtype='I', name = 'B') #INTEGER
        D = prob.addVar(lb = 0, name = 'D') #INTEGER

        #Variable to check accuracy
        V = prob.addVar(lb = 0, vtype='I', name = 'V') #INTEGER
        d_sum = prob.addVar(lb = 0, vtype='I', name = 'd_sum') #INTEGER
        p_sum = prob.addVar(lb = 0, vtype='I', name = 'p_sum') #INTEGER

        obj = h * B + g * D + 20.52
        prob.setObjective(obj, GRB.MINIMIZE)


        #---Routing---

        #TSP-1.1
        for j in range(n):
            prob.addConstr(quicksum(x[i,j] for i in range(n)) <= 1)

        #TSP1.2    
        for i in range(n):
            prob.addConstr(quicksum(x[i,j] for j in range(n)) <= 1)

        #TSP-1.3
        for i in range(n):
            prob.addConstr(quicksum(x[i,j] for j in range(n)) == quicksum(x[j,i] for j in range(n))) 

        #TSP-1.4
        for i in range(1,n):
            prob.addConstr(x[i,i] == 0)    

        #Total Distance
        prob.addConstr(quicksum(x[i,j] * d_m[i][j] for i in range(n) for j in range(n)) == D)

        #Route Continuity
        for i in range(n):
            for j in range(1,n):
                prob.addConstr(z[j] >= z[i] + 1 - 1996 * (1 - x[i,j]))


        #---Rebalancing---     

        #Empty Truck
        prob.addConstr(c[0] == 20)


        for i in range(n):
            #Capacity of the truck
            prob.addConstr(c[i] <= C)

            #pick up within ideal threshold
            prob.addConstr(p[i] <= E[i])
            prob.addConstr(p[i] >= E[i] - I[i][1])

            #drop off within ideal threshold
            prob.addConstr(d[i] <= cap[i] - E[i])
            prob.addConstr(d[i] >= (I[i][0] - E[i])) #nonbinding when surplus ???    

        #Total amount of bikes rebalanced
        prob.addConstr(quicksum(p[i] + d[i] for i in range(n)) == B)

        for i in range(1,n):
            #Pick up XOR Drop off at any station
            prob.addConstr(s1[i] * d[i] + s2[i] * p[i] == 0)

        #A station is visted when necessary (synergy with the route)
        for i in range(1,n):
            prob.addConstr(d[i] + p[i] >= quicksum(x[j,i] for j in range(n)))
            prob.addConstr(c[i] + d[i] + p[i] <= 2 * C * quicksum(x[j,i] for j in range(n)))


        #Continuity of truck inventory
        for i in range(n):
            for j in range(1,n):
                prob.addConstr(c[j] <= c[i] - d[i] + p[i] + C * (1 - x[i,j]))
                prob.addConstr(c[j] >= c[i] - d[i] + p[i] - C * (1 - x[i,j]))


        #---Variables to check accuracy---           
        prob.addConstr(quicksum(x[i,j] for i in range(n) for j in range(n)) == V)       
        prob.addConstr(quicksum(d[i] for i in range(n)) == d_sum) 
        prob.addConstr(quicksum(p[i] for i in range(n)) == p_sum) 


        prob.optimize()

        prob.printAttr('X')
        
        #Update existing bikes after rebalancing
        E = [E[i] - p[i].x + d[i].x for i in range(n)]
        after.append(E)
        costs.append(round(obj.getValue(),2))
        
        bikes_rebalanced.append(B.x)
        bikes = bikes + B.x
        
        visits.append(V.x)
        
        daily_cost += round(obj.getValue(),2)
        print(col)
#         print(E)
#         print(round(obj.getValue(),2))
    
    daily_cost = round(daily_cost,2)
    return daily_cost,costs,visits,before,after,bikes_rebalanced,bikes

### Here we simulate the rebalancing for July 8 (14 working hours).

In [ ]:
daily_cost,costs,visits,before,after,bikes_rebalanced,bikes = hour_rebalance(Verdun_net_balance20190708)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1136 rows, 401 columns and 6649 nonzeros
Model fingerprint: 0xb8a98838
Variable types: 19 continuous, 382 integer (324 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+03]
  Objective range  [3e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 184 rows and 26 columns
Presolve time: 0.01s
Presolved: 952 rows, 375 columns, 5488 nonzeros
Variable types: 17 continuous, 358 integer (306 binary)
Found heuristic solution: objective 30.0954444
Found heuristic solution: objective 22.2204444

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 22.2204 30.0954 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.222044444890e+01, best bound 2.222044444890e+01, gap 0.0000%

    Variable            X 
---------

In [ ]:
print('Operation cost for July 8th is {} dollars.'.format(daily_cost))
print('The driver travelled {} stations and rebalanced {} bikes.'.format(sum(visits),bikes))

Operation cost for July 8th is 335.21 dollars.
The driver travelled 61.0 stations and rebalanced 77.0 bikes.


In [ ]:
# Verdun_net_balance20190709 = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/net_balance2019-07-09.csv')
# Verdun_net_balance20190709 = Verdun_net_balance20190709.drop(columns=['Unnamed: 0'])

# Verdun_net_balance20190710 = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/net_balance2019-07-10.csv')
# Verdun_net_balance20190710 = Verdun_net_balance20190710.drop(columns=['Unnamed: 0'])

# Verdun_net_balance20190711 = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/net_balance2019-07-11.csv')
# Verdun_net_balance20190711 = Verdun_net_balance20190711.drop(columns=['Unnamed: 0'])

# Verdun_net_balance20190712 = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/net_balance2019-07-12.csv')
# Verdun_net_balance20190712 = Verdun_net_balance20190712.drop(columns=['Unnamed: 0'])

# Verdun_net_balance20190713 = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/net_balance2019-07-13.csv')
# Verdun_net_balance20190713 = Verdun_net_balance20190713.drop(columns=['Unnamed: 0'])

# Verdun_net_balance20190714 = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/net_balance2019-07-14.csv')
# Verdun_net_balance20190714 = Verdun_net_balance20190714.drop(columns=['Unnamed: 0'])


In [ ]:
# daily_cost,costs,visits,before,after,bikes_rebalanced,bikes = hour_rebalance(Verdun_net_balance20190714)